In [56]:
import math
import scipy
import numpy
import networkx as nx
import munkres

import pandas as pd

file = pd.read_csv('starbucks_2018_11_06.csv')

countryList = ['JP','KR','MY','FR','MX']
tempStoresData = [[row['state'], row['latitude'], row['longitude'], row['name']] for index, row in file.iterrows() if row['state'] in countryList]
storesData = pd.DataFrame(tempStoresData)
storesData.columns = ['country', 'latitude', 'longitude','name']

japanStoreLocation = []
koreaStoreLocation = []
malaysiaStoreLocation = [] 
franceStoreLocation = []
mexicoStoreLocation = []

for i in range(len(storesData)):
    if storesData['country'][i] == 'JP':
        japanStoreLocation.append((storesData['latitude'][i], storesData['longitude'][i]))
    elif storesData['country'][i] == 'KR':
        koreaStoreLocation.append((storesData['latitude'][i], storesData['longitude'][i]))
    elif storesData['country'][i] == 'MY':
        malaysiaStoreLocation.append((storesData['latitude'][i], storesData['longitude'][i]))
    elif storesData['country'][i] == 'FR':
        franceStoreLocation.append((storesData['latitude'][i], storesData['longitude'][i]))
    elif storesData['country'][i] == 'MX':
        mexicoStoreLocation.append((storesData['latitude'][i], storesData['longitude'][i]))


In [57]:
# get each country distance matrix
distanceMatrix = []
for country in countryList:
    fileS = 'distance_matrix/' + country + '.csv'
    dm = pd.read_csv(fileS)
    distanceMatrix.append(dm)
    
total_distance = [0 for i in range(len(countryList))]

In [63]:
# get the the distribution center data
file2 = pd.read_csv('Distribution Center.csv')
distribution_center = file2
# distribution_center['0'][0]

340

In [66]:
tempNode = [japanStoreLocation,koreaStoreLocation,malaysiaStoreLocation,franceStoreLocation,mexicoStoreLocation]
total_distance = []

for k in range(len(countryList)):
    currentDistanceMatrix = distanceMatrix[k]
    n = len(currentDistanceMatrix)
    G = nx.complete_graph(n)
    
    for a,b in G.edges:
        G.edges[a,b]['distance'] = currentDistanceMatrix[str(a)][b]
    
    T = nx.minimum_spanning_tree(G, weight='distance')  
    
    odd_degree_nodes = [i for i in T.nodes if T.degree(i) % 2]
    node_colors = [ T.degree(i) % 2 for i in T.nodes ]
    
    for i,j in G.edges:
        G.edges[i,j]['neg_length'] = - G.edges[i,j]['distance']
    matching = nx.max_weight_matching(G.subgraph(odd_degree_nodes), maxcardinality=True, weight='neg_length')
    
    M = nx.MultiGraph()
    M.add_nodes_from(range(n))
    M.add_edges_from(T.edges())
    M.add_edges_from(matching)
    
    initial_tour = list(nx.eulerian_circuit(M, source=distribution_center['0'][k]))
    
    tour = [ distribution_center['0'][k] ]
    for i,j in initial_tour:
        if j not in tour:
            tour.append(j)
    
    tour_edges = [ (tour[i-1], tour[i]) for i in range(n)]
    
    Total_Distance = 0
    for i in range(len(tour)-1):
        Total_Distance += currentDistanceMatrix[str(tour[i])][tour[i+1]]

    Total_Distance += currentDistanceMatrix[str(tour[i+1])][tour[0]]
    print(Total_Distance)
    total_distance.append(Total_Distance)

8336.093073178992
1956.409536989422
4441.871589199802
1393.8438162817981
11945.13488495779


In [67]:
total_distance

[8336.093073178992,
 1956.409536989422,
 4441.871589199802,
 1393.8438162817981,
 11945.13488495779]

In [ ]:
# for detailed code, look at Distribution Center Location File

# [8336.093073178992,
#  1956.409536989422,
#  4441.871589199802,
#  1393.8438162817981,
#  11945.13488495779]